# 0. Import

In [1]:
import os
import re
import shutil
import sqlite3
import pandas as pd

# 1. Copy CSVs to New Dataset Folder

In [2]:
# Make datasets folder

dataset_dir = 'datasets'

try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
    print("Folder created successfully")
except:
    print("Directory already exists or there is another error")

Folder created successfully


In [3]:
# Change directory

os.chdir("csv")
print("Directory changed to csv folder")

Directory changed to csv folder


In [7]:
# Copy CSVs into datasets folder

src_dir = "."
dst_dir = "../datasets"

try:
    for root, dirs, files in os.walk(src_dir):
        for f in files:
            if f.endswith('.csv'):
                shutil.copy(os.path.join(root,f), dst_dir)
            print(f"File '{f}' copied successfully")
except:
    print("There are no CSV files to copy")

File 'Company Client List.csv' copied successfully
File 'Company Orders.csv' copied successfully


# 2. Read Each CSV into a DataFrame

In [9]:
# Make list of CSVs

csv_files = []

for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [11]:
# Read CSVs 

data_path = '../'+dataset_dir+'/'

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
        print(f"{file} read successfully")
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")

Company Client List.csv read successfully
Company Orders.csv read successfully


In [12]:
csv_files

['Company Client List.csv', 'Company Orders.csv']

In [15]:
# Check dfs

df['Company Client List.csv'].head()

,id,company name,Contact First name,Contact Last name,Email,phone
0,1,Avamba,Fanchon,Foye,ffoye0@reddit.com,274-998-7209
1,2,Avaveo,Galen,Lambdean,glambdean1@parallels.com,346-819-3046
2,3,Flipstorm,Candace,Pantling,cpantling2@cdbaby.com,923-161-4383
3,4,Photofeed,Cullen,Rubra,crubra3@theglobeandmail.com,579-775-6516
4,5,Demivee,Adamo,Matuszinski,amatuszinski4@topsy.com,677-300-0857


In [16]:
df['Company Orders.csv'].head()

,id,Company name,Order Number,Order amount,order date,paid?
0,1,Miboo,453982,$87317.36,12/4/2020,False
1,2,Skiba,34634,$2878526.67,1/18/2022,False
2,3,Devpulse,681029,$2659083.12,2/2/2022,False
3,4,Shufflester,928473,$2139029.96,11/14/2021,True
4,5,Fliptune,915869,$250194.39,5/12/2020,True


# 3. Send Clean Version of DataFrame to Database

In [21]:
# Change directory

os.chdir("..")

os.listdir(os.getcwd())

['.git',
 '.ipynb_checkpoints',
 'csv',
 'Database Automation.ipynb',
 'datasets',
 'original_csvs',
 'README.md']

In [22]:
# Make datasets folder

database_dir = 'database'

try:
    mkdir = 'mkdir {0}'.format(database_dir)
    os.system(mkdir)
    print("Folder created successfully")
except:
    print("Directory already exists or there is another error")

Folder created successfully


In [25]:
os.chdir(database_dir)

for k in csv_files:    
    dataframe = df[k]
    
    # Clean table name
    clean_table_name = k.lower().replace(" ", "_").replace(r"/""\\", "_")
    clean_table_name = re.sub(r'[^a-zA-Z0-9_.]','', clean_table_name)    
    
    #Clean table data
    dataframe.columns = [x.lower().replace(" ", "_").replace(r"/""\\", "_").strip() for x in dataframe.columns]
    dataframe.columns = dataframe.columns.str.replace('[^A-Za-z\s_.]+', '', regex=True)
    
    table_name = '{0}'.format(clean_table_name.split('.')[0])
    
    # Change data types
    new_dtypes = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
    }
    
    col_strings = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(new_dtypes)))
    
    # Write to db with SQLite
    conn = sqlite3.connect('company')
    c = conn.cursor()

    c.execute('''
    CREATE TABLE IF NOT EXISTS table_name (
    col_strings    
    )
    ''')
    print('{0} was created successfully'.format(table_name))
    conn.commit()
    
    dataframe.to_sql(table_name, conn, if_exists='replace', index=False)
    
    conn.close()

company_client_list was created successfully
company_orders was created successfully
